Import Libraries

In [251]:
from pyvis.network import Network
import networkx as nx
import random
from igraph import Graph
from helpers import createPaper, getNewCC, getCommNodes, splitCommunity

Define initial parameters

In [252]:
# define time steps
timeSteps = 30

# Probabilities
# probability that you generate new author
probNewAuthor = 0.5
# probability that you stop at a given node
probStop = 0.7
# probability that a split event occurs
pd = 0.5

# define initial scholars, will be in form (id, scholarField, color)
scholarField = 0
nodeID = 0


Create Model

In [253]:
network = nx.Graph()
network.add_node(nodeID, label=scholarField, color="red")
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    currNodes = list(network.nodes())
    authors = [random.choice(currNodes)]

    # with probability, add new author to network set as main author with a coauthor
    if random.random() < probNewAuthor:
        # generate author and field
        nodeID += 1
        author = nodeID

        # generate random coauthor from currNodes,
        coauthorID = random.choice(currNodes)

        # add node with field being the co-author's field
        scholarField = network.nodes[coauthorID]["label"]
        network.add_node(author, label=scholarField, color="red")
        network.add_edge(author, coauthorID, weight=1, width=1)

        # update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    createPaper(network, authors, probStop)

    # split random discipline with prob pd
    if random.random() < pd:
        commNodes = getCommNodes(network, 0)
        splitCommunity(network, commNodes)

NameError: name 'Network' is not defined

Display Network

In [ ]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 1, 'width': 1}), (0, 3, {'weight': 2, 'width': 1}), (0, 4, {'weight': 1, 'width': 1}), (0, 5, {'weight': 1, 'width': 1}), (0, 6, {'weight': 2, 'width': 1}), (0, 7, {'weight': 1, 'width': 1}), (0, 15, {'weight': 1, 'width': 1}), (1, 2, {'weight': 3, 'width': 1}), (1, 11, {'weight': 1, 'width': 1}), (1, 14, {'weight': 1, 'width': 1}), (2, 11, {'weight': 1, 'width': 1}), (3, 4, {'weight': 1, 'width': 1}), (3, 9, {'weight': 1, 'width': 1}), (3, 17, {'weight': 1, 'width': 1}), (4, 8, {'weight': 1, 'width': 1}), (6, 16, {'weight': 1, 'width': 1}), (7, 12, {'weight': 1, 'width': 1}), (9, 10, {'weight': 1, 'width': 1}), (12, 13, {'weight': 1, 'width': 1}), (14, 18, {'weight': 1, 'width': 1})]
